# Identifying coordination environments

The Materials Virtual Lab have a notebook which describes the use of `ChemEnv` to identify coordination environments (http://matgenb.materialsvirtuallab.org/2018/01/01/ChemEnv-How-to-automatically-identify-coordination-environments-in-a-structure.html).

This notebook will introduce you to another tool within the `pymatgen` package which can be used to determine coordination environments: `CrystalNN`. Additionally, another tool has been built on top of CrystalNN to generate crystallographic text descriptions for material structures `Robocrystallographer`.

## Installation

In [1]:
import pprint

from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.local_env import CrystalNN
mpr = MPRester()

## Query the Materials Project (MP) for a structure
Let's load a crystal structure from the materials project

In [2]:
struct = mpr.get_structure_by_material_id("mp-7000")
print(struct)

Full Formula (Si3 O6)
Reduced Formula: SiO2
abc   :   5.021503   5.021503   5.510570
angles:  90.000000  90.000000 120.000008
Sites (9)
  #  SP           a         b         c    magmom
---  ----  --------  --------  --------  --------
  0  Si    0.523695  0.523695  0                0
  1  Si    0         0.476305  0.666667         0
  2  Si    0.476305  0         0.333333         0
  3  O     0.256094  0.414854  0.794543        -0
  4  O     0.585146  0.84124   0.127876        -0
  5  O     0.15876   0.743906  0.46121         -0
  6  O     0.414854  0.256094  0.205457        -0
  7  O     0.743906  0.15876   0.53879         -0
  8  O     0.84124   0.585146  0.872124        -0


Here, we have the structure of alpha-SiO<sub>2</sub>.
![mp-7000_SiO2](SiO2_mp-7000_annotated.png)

 The graphic is created with VESTA: K. Momma and F. Izumi, "VESTA 3 for three-dimensional visualization of crystal, volumetric and morphology data," *J. Appl. Cryst.,* **2011**, *44*, 1272.

## Alternatively: Use the new materials project API, `mp_api`

In [3]:
# Remove the comment
#from mp_api.client import MPRester 
#with MPRester() as mpr:
#    struct = mpr.get_structure_by_material_id("mp-7000")

## Using CrystalNN

The `pymatgen.analysis.local_env` module contains many classes which can perform analyses of local environments. For this tutorial, we will just focus on [`CrystalNN`](https://pymatgen.org/pymatgen.analysis.local_env.html#pymatgen.analysis.local_env.CrystalNN).

We will demonstrate a very straightforward method for finding coordination environments.


In [4]:
# Initialise CrystalNN with default parameters
cnn  = CrystalNN()

# Iterate over the structure to find the coordination number of each site
for i, n in enumerate(struct):
    print(f' Site {i} contains the element, {n.species_string}, which has a coordination number of {cnn.get_cn(struct, i)} \n')

 Site 0 contains the element, Si, which has a coordination number of 4 

 Site 1 contains the element, Si, which has a coordination number of 4 

 Site 2 contains the element, Si, which has a coordination number of 4 

 Site 3 contains the element, O, which has a coordination number of 2 

 Site 4 contains the element, O, which has a coordination number of 2 

 Site 5 contains the element, O, which has a coordination number of 2 

 Site 6 contains the element, O, which has a coordination number of 2 

 Site 7 contains the element, O, which has a coordination number of 2 

 Site 8 contains the element, O, which has a coordination number of 2 



/Users/aonwu/.conda/envs/atomate2_env/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/Users/aonwu/.conda/envs/atomate2_env/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


From the above output, we can see it is quite straightforward to obtain the coordination number of the sites within a material. However, the output is quite redundant as each O  in SiO<sub>2</sub> is equivalent. Likewise for Si. For a much larger unit cell, this output would not be that helpful. 

We can solve this issue in two ways:
- Using symmetry by using spglib
- Representing the crystal as a graph


### Using symmetry: Interfacing to spglib

The [`pymatgen.symmetry.analyzer`](https://pymatgen.org/pymatgen.symmetry.analyzer.html) module acts as an interface to `spglib`.

We will be using the `SpacegroupAnalyzer` class to determine the equivalent sites in alpha-SiO<sub>2</sub>

In [5]:
# Import the class
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


sga = SpacegroupAnalyzer(struct)

# Get the equivalent sites in the structure
equivalent_sites = sga.get_symmetry_dataset()["equivalent_atoms"]

unique_sites = set(equivalent_sites)

for i in unique_sites:
    print(f' There are {list(equivalent_sites).count(i)} equivalent {struct[i].species_string} atoms in SiO2')

 There are 3 equivalent Si atoms in SiO2
 There are 6 equivalent O atoms in SiO2


With this consideration, we can re-do our coordination number analysis for only the inequivalent sites within the structure

In [6]:
for i in unique_sites:
    print(f'{struct[i].species_string} has a coordination number of {cnn.get_cn(struct, i)}')

Si has a coordination number of 4
O has a coordination number of 2


### Structures as a graph 
The [`pymatgen.analysis.graphs`](https://pymatgen.org/pymatgen.analysis.graphs.html) module enables the usage of a graph representation of the crystal.

`CrystalNN` has a method (inherited from its `NearNeighbors` base class), [`get_bonded_structure`](https://pymatgen.org/pymatgen.analysis.local_env.html#pymatgen.analysis.local_env.NearNeighbors.get_bonded_structure) which takes in a pymatgen `Structure` object and returns a `StructureGraph` object.

The `StructureGraph` has a method, [`types_of_coordination_environments`](https://pymatgen.org/pymatgen.analysis.graphs.html#pymatgen.analysis.graphs.StructureGraph.types_of_coordination_environments), which returns a list of the coordination environments present in the `StructureGraph`.

In [7]:
# Get the bonded structure - this produces a StructureGraph object
bonded=cnn.get_bonded_structure(struct)

# This will output the coordination environments present in the structure
print(bonded.types_of_coordination_environments())


['O-Si(2)', 'Si-O(4)']


The above output shows that there are two different coordination environments in alpha-SiO<sub>2</sub>:

- 'O-Si(2)': This output indicates that O in alpha-SiO<sub>2</sub> is bonded to two other Si atoms and hence has a coordination number of 2.
- 'Si-O(4)': This output indicates that Si in alpha-SiO<sub>2</sub> is bonded to four other O atoms and hence has a coordination number of 4.

## Using Robocrystallographer

[Robocrystallographer](https://hackingmaterials.lbl.gov/robocrystallographer/index.html) is a tool which can be used to generate text descriptions of structures. 

In [8]:
# Imports
# Uncomment this line if robocrys has not been installed
# !pip install robocrys
from robocrys import StructureCondenser, StructureDescriber
import pprint

condenser = StructureCondenser()
describer = StructureDescriber()

condensed_structure = condenser.condense_structure(struct)
description = describer.describe(condensed_structure)

#pprint.pprint(condensed_structure)
print(description)

Reading file /Users/aonwu/.conda/envs/atomate2_env/lib/python3.8/site-packages/robocrys/condense/mineral_db.json.gz: 0it [00:00, ?it/s]2     | 77/180 [00:00<00:00, 766.92it/s]
Decoding objects from /Users/aonwu/.conda/envs/atomate2_env/lib/python3.8/site-packages/robocrys/condense/mineral_db.json.gz: 100%|##########| 180/180 [00:00<00:00, 1075.00it/s]
/Users/aonwu/.conda/envs/atomate2_env/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/Users/aonwu/.conda/envs/atomate2_env/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
/Users/aonwu/.conda/envs/atomate2_env/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified o

SiO2 is quartz (alpha) structured and crystallizes in the trigonal P3_121 space group. Si(1) is bonded to four equivalent O(1) atoms to form corner-sharing SiO4 tetrahedra. All Si(1)–O(1) bond lengths are 1.63 Å. O(1) is bonded in a bent 150 degrees geometry to two equivalent Si(1) atoms.


The above output provides a description of the crystal provides a lot of important information. A lot of this information is available in the intermediate `condensed_structure` variable which is a JSON representation of the structure.

In [9]:
# Print the JSON format
pprint.pprint(condensed_structure)

{'angles': {0: {0: {'corner': [146.72642164417323,
                               146.72642164417329,
                               146.72642164417329,
                               146.72642164417329]}},
            3: {3: {'corner': [108.33914884657402,
                               109.19814528445058,
                               110.52931187496212,
                               110.52931187496212,
                               109.19814528445052,
                               109.04217117173853]}}},
 'component_makeup': [0],
 'components': {0: {'dimensionality': 3,
                    'formula': 'SiO2',
                    'molecule_name': None,
                    'orientation': None,
                    'sites': [0, 0, 0, 3, 3, 3, 3, 3, 3]}},
 'crystal_system': 'trigonal',
 'dimensionality': 3,
 'distances': {0: {3: [1.625527396842719,
                       1.6255273968427193,
                       1.6284404722925014,
                       1.6284404722925014]},
       

The information that we are interested in is in the 'sites' key

In [10]:
pprint.pprint(condensed_structure['sites'])

{0: {'element': 'Si',
     'geometry': {'likeness': 0.9940319496362562, 'type': 'tetrahedral'},
     'nn': [3, 3, 3, 3],
     'nnn': {'corner': [0, 0, 0, 0]},
     'poly_formula': 'O4',
     'sym_labels': (1,)},
 3: {'element': 'O',
     'geometry': {'likeness': 0.9713927441125383, 'type': 'bent 150 degrees'},
     'nn': [0, 0],
     'nnn': {'corner': [3, 3, 3, 3, 3, 3]},
     'poly_formula': None,
     'sym_labels': (1,)}}


In the above output, the keys, 0 and 3, refer to the unique sites in the structure. Within the nested dictionaries, the 'geometry' key contains another nested dictionary where the key 'type' provides information about the coordination environment.